In [ ]:
# Random 500 sentences 
# Annotate with 0.0, 0.5, 1 (negativ, neutral, positiv) only the sentiment (meaning is + or -) 
# Ratio interal/external

In [1]:
# Use ChatGTP API to annoatete with zero-shot or few-show prompts:

